In [5]:
reset()
%display latex
Partitions.options.latex="list"
from sage.misc.cachefunc import cached_method, cached_function
from sage.structure.unique_representation import UniqueRepresentation
from itertools import chain, combinations

load("partition_4d.sage")

KT_vars = ['x1','x2','x3','x4','y1','y2','y3','y4','y5','y6','y7','y8','y9',
           'v1','v2','v3','v4','v5','v6','v7','v8','v9']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

sqrtKT_vars = ['sx1','sx2','sx3','sx4','sy1','sy2','sy3','sy4','sy5','sv1','sv2','sv3','sv4','sv5']
sqrtKT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t1','t2','t3','t4','m1','m2','m3','m4','m5','m6','m7','m8','m9',
             'w1','w2','w3','w4','w5','w6','w7','w8','w9']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


rank=1

x1,x2,x3,x4,y1,y2,y3,y4,y5,y6,y7,y8,y9,v1,v2,v3,v4,v5,v6,v7,v8,v9=KT_ring.gens()
#sx1,sx2,sx3,sx4,sy1,sy2,sy3,sy4,sy5,sv1,sv2,sv3,sv4,sv5=sqrtKT_ring.gens()
t1,t2,t3,t4,m1,m2,m3,m4,m5,m6,m7,m8,m9,w1,w2,w3,w4,w5,w6,w7,w8,w9= coho_ring.gens()
b=var('b')
v=[v1,v2,v3,v4,v5,v6,v7,v8,v9]
w=[w1,w2,w3,w4,w5,w6,w7,w8,w9]
y=[y1,y2,y3,y4,y5,y6,y7,y8,y9]

pP4ds = Partitions4d()

def character(p):
    Nx1, Nx2, Nx3, Nx4 = p._Nx1, p._Nx2, p._Nx3, p._Nx4
    return sum(x1**i * x2**j * x3**k * x4**l
           for (i,j,k,l) in p.boxes(Nx1=Nx1, Nx2=Nx2, Nx3=Nx3, Nx4=Nx4))

def dual(f):
    if f in ZZ:
        return f
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def measure_unsymmetrized(f, inv=True):
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom

def specialize(f):
    if f in ZZ:
        return f
    return f(x4=x1^-1*x2^-1*x3^-1)

def degn(f,n,b=b):
    g=f.taylor(b,0,n)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

def deg0(f):
    if f==0: 
        return 0
    return pretty(degn(f(t1=b*t1,t2=b*t2,t3=b*t3,t4=b*t4,
                         m1=b*m1,m2=b*m2,m3=b*m3,m4=b*m4,m5=b*m5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0))

def deg0n(f,n):
    if f==0: 
        return 0
    return pretty(degn(f(t1=b*t1,t2=b*t2,t3=b*t3,t4=b*t4,
                         m1=b*m1,m2=b*m2,m3=b*m3,m4=b*m4,m5=b*m5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),n))


def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[list(pP4ds.with_num_boxes(ptn[i]))]
        total+=[p for p in itertools.product(*lst)]
    return total


def sign(Z):
    s=1
    for i in range(len(Z)):
        s=s*Z[i].sign()
    return s
        

def Nvir(Z):
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            Qj=character(Z[j])
            Qiinv=dual(character(Z[i]))
            total+=(Qj+Qiinv/(x1*x2*x3*x4)-Qj*Qiinv*(1-x1)*(1-x2)*(1-x3)*(1-x4)/(x1*x2*x3*x4))*(y[j]/y[i])
    return total

def sqrtNvir(Z,check=False):
    sqrtN=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            Qj=specialize(character(Z[j]))
            Qi=specialize(character(Z[i]))
            Qiinv=dual(Qi)
            sqrtN+=(Qj-(1-x1^-1)*(1-x2^-1)*(1-x3^-1)*Qiinv*Qj)*(y[j]/y[i])
        

    if check:
        if sqrtN+dual(sqrtN)!=specialize(Nvir(Z)):
            print(sqrtN)
            print(specialize(Nvir(Z)))
            raise ValueError("something went wrong in computation"+Z.__str__())

    return sqrtN+x1^-1*x2^-1+x3^-1*x2^-1+x1^-1*x3^-1

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)
    
def from_monomial(m):
    R = m.parent()
    if R== KT_ring:
        return sum(m.degree(R.gen(i)) * coho_ring.gen(i) for i in range(len(coho_vars)))
    elif R==sqrtKT_ring:
        return sum((1/2)*m.degree(R.gen(i)) * coho_ring.gen(i) for i in range(len(coho_vars)))
    else:
        raise NotImplementedError("%s" % R)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return coho_ring.fraction_field()(prod(numer) / prod(denom))


def total_chern_quot(Z, rank):
    total=1
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(y[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r) for r in roots)
    return total


def Chern_Num_quot(n,N,r):
    total=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*top_chern(-sqrtNvir(Z))*total_chern_quot(Z,r)
        total+=add
    return total

def cut_off_n(f,n):
    if f in ZZ:
        return f
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)>n:
            del D[exp]
    return coho_ring(D)

def restrict(f,n):
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)!=n:
            del D[exp]
    return coho_ring(D)

def total_segre_quot(Z, rank,n):
    total=1
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(y[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total*=prod(1/(1+r) for r in roots)
    return total

def total_segre_quot_fast(Z, rank,n,N):
    total=1
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(y[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        for r in roots:
            total=cut_off_n(total*(1+sum((-1)^(k+1)*r^(k+1) for k in range(n*N))),n*N)
    return restrict(total,n*N)

def Segre_Num_quot_fast(n,N,r):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*top_chern(-sqrtNvir(Z))*total_segre_quot_fast(Z,r,n,N)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Segre_Num_quot(n,N,r):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*top_chern(-sqrtNvir(Z))*total_segre_quot(Z,r,n)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Segre_Num_quot_sym(n,N,r):
    total=0
    maxcounter=len(fixed_locus(n,r))
    counter=0
    for Z in fixed_locus(n,r):
        add=sign(Z)*symmetrize(top_chern(-sqrtNvir(Z)))*symmetrize(total_segre_quot(Z,N,n))
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def determinant_quot(Z,n, rank):
    det=1
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(y[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def determinant_fast(f,n):
    if f in ZZ:
        return 1
    else:
        total=1
        for coeff,mon in f:
            term=1+sum((coeff*from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n))
            total=cut_off_n(total*term,n)
        return total

def determinant_quot_fast(Z,n, rank,N):
    det=1
    v_1=1
    v_2=1
    Q_2=1
    for i in range(rank):
        v_1=cut_off_n(v_1*(1+sum((w[i])^(k+1)/factorial(k+1) for k in range(n*N))),n*N)
    for i in range(n):
        v_2=cut_off_n(v_2*v_1,n*N)
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(y[i])
        det = cut_off_n(det* determinant_fast(Q,n*N),n*N)
    for i in range(rank):
        Q_2=cut_off_n(Q_2*det,n*N)
    return cut_off_n(Q_2*v_2,n*N)

def todd_class_fast(f,n):
    R = f.parent()
    total=1
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)
    for coeff, mon in L:
        t=1
        if mon==1:
            term = 0
        if coeff < 0:
            term = 1+sum((-1)^(k+1)*(from_monomial(mon))^(k+1)/factorial(k+2) for k in range(n))
            for i in range(-coeff):
                t=cut_off_n(t*term,n)
        elif coeff > 0:
            term = 1+sum(bernoulli_polynomial(1, k+1)*(from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n))
            for i in range(coeff):
                t=cut_off_n(t*term,n)
        total=cut_off_n(total*t,n)
    return total

def Verlinde_Num_quot_fast(n,N, r):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*top_chern(-sqrtNvir(Z))*\
            restrict(todd_class_fast(sqrtNvir(Z),n*N)*determinant_quot_fast(Z,n,r,N),n*N)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Verlinde_Num_quot(n,N, r):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*measure_unsymmetrized(sqrtNvir(Z))*\
            determinant_quot(Z,n,r)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Verlinde_Num_quot_inv(n,N, r):
    total=0
    for Z in fixed_locus(n,N):
        add=sign(Z)*measure_unsymmetrized(sqrtNvir(Z))*\
            (1/determinant_quot(Z,n,r))
        total+=add
    return total

def set_exp(f,n):
    g=cut_off_n(f(x1=1+sum(t1^(k+1)/factorial(k+1) for k in range(n)),
        x2=1+sum(t2^(k+1)/factorial(k+1) for k in range(n)),
        x3=1+sum(t3^(k+1)/factorial(k+1) for k in range(n)),
        x4=1+sum(t4^(k+1)/factorial(k+1) for k in range(n)),
        y1=1+sum(m1^(k+1)/factorial(k+1) for k in range(n)),
        y2=1+sum(m2^(k+1)/factorial(k+1) for k in range(n)),
        y3=1+sum(m3^(k+1)/factorial(k+1) for k in range(n)),
        y4=1+sum(m4^(k+1)/factorial(k+1) for k in range(n)),
        y5=1+sum(m5^(k+1)/factorial(k+1) for k in range(n)),
        v1=1+sum(w1^(k+1)/factorial(k+1) for k in range(n)),
        v2=1+sum(w2^(k+1)/factorial(k+1) for k in range(n)),
        v3=1+sum(w3^(k+1)/factorial(k+1) for k in range(n)),
        v4=1+sum(w4^(k+1)/factorial(k+1) for k in range(n)),
        v5=1+sum(w5^(k+1)/factorial(k+1) for k in range(n))),n)
    return g

def invert_exp(f):
    g=f(t1=-t1,t2=-t2,t3=-t3,t4=-t4,
                  m1=-m1,m2=-m2,m3=-m3,m4=-m4,m5=-m5,
                  w1=-w1,w2=-w2,w3=-w3,w4=-w4,w5=-w5)
    return g

def symmetrize(f):
    if f in ZZ:
        return f
    return f(m1=w1,m2=w2,m3=w3,m4=w4,m5=w5,w1=m1,w2=m2,w3=m3,w4=m4,w5=m5)

def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(y1=v1,y2=v2,y3=v3,y4=v4,y5=v5,v1=y1,v2=y2,v3=y3,v4=y4,v5=y5)


def check_sym(nlimit):
    with open("symmetry4d-segre-checked.txt") as f:
        lines_segre = f.read().splitlines()
    for n in range(1,nlimit+1):
        N,r=[2,2]
        print("checking segre sym: n="+str(n)+", N="+str(N)+", r="+str(r))
        if str(n)+','+str(N)+','+str(r) in lines_segre:
            print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
        else:
            print("computing S1")
            A=Segre_Num_quot(n,N,r)
            print("computing S2")
            B=Segre_Num_quot_sym(n,N,r)
            A1=A.numerator()
            B1=B.numerator()
            A2=A.denominator()
            B2=B.denominator()
            print("computing diff")
            diff=(A1.lc()/A2.lc())/(B1.lc()/B2.lc())
            A1=A1*(B1.lc()/A1.lc())
            A2=A2*(B2.lc()/A2.lc())
            if diff==(-1)^(n*(r+N)) and A1==B1 and A2==B2:
                writeline("symmetry4d-segre-checked.txt",str(n)+','+str(N)+','+str(r))
                print("checked")
            else:
                print("failed")
                    
                
def clear_all_files():
    open('SV4d-checked.txt', 'w').close()
    open('symmetry4d-segre-checked.txt', 'w').close()
    open('symmetry4d-Verlinde-checked.txt', 'w').close()
    
    
def SV_check_fast(nlimit, rlimit):
    with open("SV4d-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        for N in range(1,rlimit+1):
            for r in range(1,rlimit+1):
                print("n="+str(n)+", N="+str(N)+", r="+str(r))
                if str(n)+','+str(N)+','+str(r) in lines:
                    print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
                    continue
                print("computing S1")
                S1=Segre_Num_quot_fast(n,N,r)
                print("computing V1")
                V1=Verlinde_Num_quot_fast(n,N,r)
                print("computing diff")
                diff= S1-(-1)^(N*n)*V1
                check1=false
                check2=false
                check3=false
                if diff.numerator()==0:
                    check1=true
                    check2=true
                    check3=true
                else:
                    for f,e in list(diff.numerator().factor()):
                        if e>=2:
                            if f==t1+t2:
                                check1=true
                            if f==t2+t3:
                                check2=true
                            if f==t1+t3:
                                check3=true
                if check1 and check2 and check3:
                    writeline("SV4d-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    

def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()

In [6]:
check_sym(3)

checking segre sym: n=1, N=2, r=2
n=1, N=2, r=2 already checked
checking segre sym: n=2, N=2, r=2
n=2, N=2, r=2 already checked
checking segre sym: n=3, N=2, r=2
computing S1
added1/28
added2/28
added3/28
added4/28
added5/28


KeyboardInterrupt: 

In [2]:
A=Segre_Num_quot_sym(3,1,3)
A1=A.numerator()
A2=A.denominator()

added1/55
added2/55
added3/55
added4/55
added5/55
added6/55
added7/55
added8/55
added9/55
added10/55
added11/55
added12/55


KeyboardInterrupt: 

In [3]:
writeline("Segre-313-num.txt",A1.__str__())
writeline("Segre-313-den.txt",A2.__str__())